In [1]:
import os
import shutil
import random
import h5py
import json
from collections import Counter
from tqdm import tqdm

In [2]:
# indoor_vg = h5py.File("/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG/final/VG-SGG.h5")
indoor_vg_dict = json.load(open("/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/VG-SGG-dicts.json"))
image_data = json.load(open('/home/maelic/Documents/Datasets/VG/anno/image_data.json'))
objects = json.load(open("/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/objects.json"))

h5_file = h5py.File("/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/VG-SGG.h5", "r")

In [3]:
object_to_idx = indoor_vg_dict['idx_to_label']

with open('/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno/classes.txt', 'w') as f:
    for key,v in object_to_idx.items():
        f.write(v + '\n')

In [4]:
object_to_idx = indoor_vg_dict['label_to_idx']
dest_folder = '/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno/labels'
image_dest_folder = '/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno/images'
annotated_images = h5_file['split_rel']

if not os.path.exists(dest_folder):
    os.mkdir(dest_folder)

if not os.path.exists(image_dest_folder):
    os.mkdir(image_dest_folder)

for idx, img in enumerate(tqdm(objects)):
    if annotated_images[idx] == -1:
        continue
    img_width = image_data[idx]['width']
    img_height = image_data[idx]['height']
    image_id = image_data[idx]['image_id']
    for obj in img['objects']:
        obj_name = obj['name']
        obj_x = obj['x']
        obj_y = obj['y']
        obj_width = obj['w']
        obj_height = obj['h']
        obj_cat = object_to_idx[obj_name]
        # copy image to dest folder
        shutil.copyfile('/home/maelic/Documents/Datasets/VG/VG_100K/'+str(image_id)+'.jpg', os.path.join(image_dest_folder, str(image_id)+'.jpg'))
        with open(os.path.join(dest_folder, str(image_id) + '.txt'), 'a') as f:
            xcenter = (obj_x + obj_width/2) / img_width
            ycenter = (obj_y + obj_height/2) / img_height
            w = obj_width / img_width
            h = obj_height / img_height
            obj_cat = obj_cat - 1
            f.write(str(obj_cat) + ' ' + str(xcenter) + ' ' + str(ycenter) + ' ' + str(w) + ' ' + str(h)+'\n')

  0%|          | 0/108073 [00:00<?, ?it/s]

100%|██████████| 108073/108073 [01:08<00:00, 1585.33it/s]


In [6]:
from sklearn.model_selection import train_test_split

base_path = '/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno/'
dest_folder = os.path.join(base_path,'labels')
image_dest_folder = os.path.join(base_path,'images')

images = [os.path.join(image_dest_folder, x) for x in os.listdir(image_dest_folder)]
annotations = [os.path.join(dest_folder, x) for x in os.listdir(dest_folder) if x[-3:] == "txt"]

images.sort()
annotations.sort()
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
# val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

root_path = 'images/'
folders = [base_path+'train',base_path+'val'] #,base_path+'test'
for folder in folders:
    os.makedirs(os.path.join(folder,root_path))
    
root_path = 'labels/'
for folder in folders:
    os.makedirs(os.path.join(folder,root_path))    

def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

move_files_to_folder(train_images, base_path+'train/images/')
move_files_to_folder(val_images, base_path+'val/images/')
# move_files_to_folder(test_images, base_path+'test/images/')
move_files_to_folder(train_annotations, base_path+'train/labels/')
move_files_to_folder(val_annotations, base_path+'val/labels/')
# move_files_to_folder(test_annotations, base_path+'test/labels/')

In [7]:
# write the yaml file
classes = indoor_vg_dict['label_to_idx'].keys()

train_path = base_path+'train/'
val_path = base_path+'val/'
# test_path = base_path+'/test/'

n_classes = len(classes)

with open(base_path+'/train.yaml', 'w') as f:
    f.write('train: '+train_path+'\n')
    f.write('val: '+val_path+'\n')
    f.write('test: '+val_path+'\n')
    f.write('nc: '+str(n_classes)+'\n')
    f.write('names: '+str(list(classes)))